In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [2]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [3]:
fea_number=8
data=pd.read_csv('./BayesianRidge_Pre_0/Input_C_049.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_047', 'Input_C_048', 'Input_C_137', 'Input_C_044',
       'Input_C_041', 'Input_C_136', 'Input_C_043', 'Input_A4_011'],
      dtype=object)

## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,8)

train_rmse: 0.00022172789652128146
test_rmse: 0.00023546602040798217
test_r2: 0.3190337509979494


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,7e-06)

test_rmse_ave: 0.00022505173779003708
[0.00023712937718824043, 0.00024464335206266505, 0.00019398886669105128, 0.0002498756010646379, 0.0002084259236082467, 0.0002213630122603428, 0.00024285143796758351, 0.0002134882864157711, 0.000204171679208436, 0.000234579841433396]


test_r2_ave: 0.3328288650927885
[0.31955912971793454, 0.402324838111494, 0.3150159413336382, 0.3552543226233065, 0.46954516087787335, 0.406098117201805, 0.25103055637760296, 0.12354277483104381, 0.38417710957315665, 0.30174070028002953]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.005],0.01)

test_rmse_ave: 0.0002334562317344059
[0.0002474261109276999, 0.0002676054513932509, 0.00019495238154806539, 0.0002684603591224828, 0.00022497666705178292, 0.00023123371853646016, 0.00024082689531972214, 0.00020479139538548885, 0.00021187269274605564, 0.0002424166453130505]


test_r2_ave: 0.289004584499655
[0.25918336153952937, 0.284864589884112, 0.3081946080047384, 0.25578045893280477, 0.3819552759243223, 0.35195238492842806, 0.26346614377713895, 0.19349691459101115, 0.33684538106087103, 0.25430672635359397]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,8,model)

train_rmse: 0.0002465000900666068
test_rmse: 0.000256249934256984
test_r2: 0.18580959310929768


## XGB

In [8]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 180, 
              max_depth = 2, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.6,
              reg_alpha =  0,
              reg_lambda = 0.1)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.00023163132541647048
[0.00024178403422652492, 0.0002605923951641053, 0.0001872796957579155, 0.00026425329340850544, 0.00022458294516532502, 0.00023883934298741461, 0.0002402783780101907, 0.00020562364020547337, 0.00020683579689405375, 0.00024624373234519647]


test_r2_ave: 0.30000012558263534
[0.29258394941076415, 0.3218561218577989, 0.36157741384780706, 0.27892315002223667, 0.38411660945697745, 0.30862077439290325, 0.26681744291925935, 0.18692855400501984, 0.36800122818531544, 0.2305760117282707]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 60,
            max_depth = 9,
            min_samples_split = 2,
            max_leaf_nodes = 10,
            min_samples_leaf =5,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.0002320167707026086
[0.0002289979038397525, 0.0002522624481159855, 0.00019535913352243056, 0.00025549552284764664, 0.00020496587873325355, 0.0002361910154974464, 0.00025281709670211335, 0.0002283193813180385, 0.0002120616160595685, 0.00025369771038985024]


test_r2_ave: 0.2876841704003815
[0.36542540091539866, 0.36451751880399363, 0.3053048073402945, 0.3259263913345103, 0.4870109630952959, 0.32386823936672027, 0.1882998608790586, -0.002462604389031009, 0.33566220616670595, 0.1832889204908682]


In [10]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=7e-06)
elastic_mod=ElasticNet(alpha=[0.005], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 180, 
              max_depth = 2, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.6,
              reg_alpha =  0,
              reg_lambda = 0.1)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 60,
            max_depth = 9,
            min_samples_split = 2,
            max_leaf_nodes = 10,
            min_samples_leaf =5,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.00022397035762692236
[0.00023222520916756054, 0.0002437337569171048, 0.00018653927664675788, 0.0002595276184302717, 0.0002113613776281038, 0.00022049743228206827, 0.00024263633422993291, 0.000206442782446151, 0.00020290854572079653, 0.00023383124280047614]


test_r2_ave: 0.34212618560793495
[0.3474130338886259, 0.4067609429770319, 0.3666155025785467, 0.3044827617003637, 0.4544981745895913, 0.4107336207467722, 0.252356756286643, 0.18043759043064012, 0.39177326918144906, 0.3061902036996851]


In [11]:

stack_mod = StackingRegressor(regressors=[vote_mod, lr, elastic_mod, random_mod], 
                           meta_regressor=lasso_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.00022505173779003708
[0.00023712937718824043, 0.00024464335206266505, 0.00019398886669105128, 0.0002498756010646379, 0.0002084259236082467, 0.0002213630122603428, 0.00024285143796758351, 0.0002134882864157711, 0.000204171679208436, 0.000234579841433396]


test_r2_ave: 0.3328288650927885
[0.31955912971793454, 0.402324838111494, 0.3150159413336382, 0.3552543226233065, 0.46954516087787335, 0.406098117201805, 0.25103055637760296, 0.12354277483104381, 0.38417710957315665, 0.30174070028002953]


In [12]:
ave(skew_data[title],y,0.15,stack_mod,0.7,vote_mod,0.15,lasso_mod)

test_rmse_ave: 0.00022374698531722235
[0.00023326709350233242, 0.00024373998438166747, 0.00018817293433113111, 0.0002562764186045101, 0.0002100359281820916, 0.00021975327681307907, 0.00024221130900678014, 0.0002077943254657319, 0.0002028749100751017, 0.00023334367280979806]


test_r2_ave: 0.34272707517866846
[0.34154420182646816, 0.4067306277474332, 0.35547292263493446, 0.32179962128434003, 0.4613184179093719, 0.4147043322328846, 0.25497375032938474, 0.16967141387225837, 0.391974900937304, 0.30908056301230524]
